In [ ]:
import numpy as np
import pylab 
import copy
from qiskit import BasicAer
from qiskit.aqua.algorithms import NumPyMinimumEigenSolver, VQE
from qiskit. aqua.components.optimizers import SLSQP
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.drivers import PySCFDriver
from qiskit.chemistry.core import Hamiltonian, QubitMappingType
from qiskit.chemistry.components.initial_states import HartreeFrok